In [90]:
import numpy as np
import pandas as pd
import string

df = pd.read_csv('emails.csv')

In [91]:
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [92]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [93]:
import nltk
import re

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FALCON\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
df['text'] = df['text'].map(lambda text:re.sub('[^a-zA-Z-\']{1,}', ' ',text))
df.text = [x.strip('Subject') for x in df.text]
df['text'] = df['text'].apply(lambda x: x.split())

In [104]:
df.head()

,text,spam
0,"[naturally, irresistible, corporate, identity,...",1
1,"[stock, trade, gunslinger, fanny, merrill, muz...",1
2,"[unbelievable, new, home, make, easy, want, sh...",1
3,"[color, print, special, request, additional, i...",1
4,"[money, get, software, software, compatibility...",1


In [95]:
df['text']=df['text'].apply(lambda text_list: list(filter(lambda text:text not in stop,text_list)))

In [96]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
ltr = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FALCON\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [97]:
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'n'),text_list)))
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'v'),text_list)))
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'a'),text_list)))
df['text']=df['text'].apply(lambda text_list: list(map(lambda word:ltr.lemmatize(word,'r'),text_list)))
df["text"] = df['text'].apply(lambda rem : (list(filter(lambda x : len(x) > 2 ,rem))))

In [106]:
df.head(10)

,text,spam
0,"[naturally, irresistible, corporate, identity,...",1
1,"[stock, trade, gunslinger, fanny, merrill, muz...",1
2,"[unbelievable, new, home, make, easy, want, sh...",1
3,"[color, print, special, request, additional, i...",1
4,"[money, get, software, software, compatibility...",1
5,"[great, nnews, hello, welcome, medzonline, gro...",1
6,"[hot, play, motion, homeland, security, invest...",1
7,"[save, money, buy, get, thing, try, cialls, ye...",1
8,"[undeliverable, home, base, business, grownup,...",1
9,"[save, money, buy, get, thing, try, cialls, ye...",1


In [98]:
corpus = df['text'].apply(' '.join)

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=10,max_df=400)
X = cv.fit_transform(corpus).toarray()
Y = df.iloc[:, 1].values

x=pd.DataFrame(X)
vocab_size = x.shape[1]
d = {'Spam': Y}
y=pd.DataFrame(data = d)
full_matrix = pd.merge(x,y,left_index=True,right_index=True,how='inner')

In [100]:
matrix=np.zeros([full_matrix.shape[0],full_matrix.shape[1]-1])
for i in range(full_matrix.shape[0]):
    matrix[i]=full_matrix.iloc[i,:-1]
#print(matrix)

In [101]:
def split_(data,label,frac):
    for i in range(data.shape[1]):
        x=data[:,i]
        data[:,i]=(x-min(x))/(max(x)-min(x)) 
    #print(data.shape)    
    b=np.ones([data.shape[0],1])
    label_=np.array(label).reshape(-1,1)
    data_final=np.hstack((b,data))
    data_final=np.hstack((data_final,label_))
    idx=np.random.randint(0,data_final.shape[0],size=int(frac*data_final.shape[0]))
    test=data_final[idx,:]
    train=np.delete(data_final,idx,axis=0)
    return train,test

label= full_matrix.ix[:,'Spam'].to_list()
train,test=split_(matrix,label,.3)

C:\Users\FALCON\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [102]:
def sgd_predict(w,x):
    y= np.dot(w,x)
    return y

def sgd_gradiant(train,lmd,T):
    theta=np.zeros(train.shape[1]-1)
    w=theta
    w_t=np.zeros([T,len(w)])
    
    for t in range(1,T):
        w=np.divide(theta,lmd*t)
        for i in range(train.shape[0]):
            y_est=sgd_predict(w,train[i,:-1])
            if train[i,-1]*y_est<1:
                theta=theta+np.multiply(train[i,-1],train[i,:-1])
            else: pass
            w_t[t]=w 
    return np.average(w_t,axis=0)

for i in range(train.shape[0]):
    if train[i,-1]==0:
        train[i,-1]=-1
    else:pass    
for i in range(test.shape[0]):
    if test[i,-1]==0:
        test[i,-1]=-1
    else:pass        
#print(train)    
w=sgd_gradiant(train,2,1000)

pr_out=[round(sgd_predict(w,test[i,:-1]),3) for i in range(test.shape[0])]
ac_out=[test[i,-1] for i in range(test.shape[0])]

def accuracy(predicted,actual):
    count=0
    for i,j in zip(predicted,actual):
        if i>0 and j>0:
            count+=1
        elif i<0 and j<0:
            count+=1
        else:pass
    return (count/len(actual))*100    
    


In [103]:
print('Accuracy of SVM model: ', accuracy(pr_out,ac_out))

Accuracy of SVM model:  91.27634660421545
